In [ ]:
f = open('bạn bè.txt','r')
sentences = f.readlines()

In [ ]:
import re

questions = []
answers = []
for sent in sentences:
  q_a = sent.split('__eou__')
  for i in range(len(q_a)):
    q_a[i] = re.sub('^ | $','',q_a[i])
  try:
    questions.append(q_a[0])
    answers.append(q_a[1])
  except Exception:
    continue

print(len(questions))
print(len(answers))
print(questions[0:5])
print(answers[0:5])

610
610
['Thích đánh lộn không?', 'Solo yasua không', 'Mai đi picnic không?', 'Mai học ca mấy vậy?', 'Còn tiền không?']
['Ngon nhà vô', 'Chấp lun 2 mạng đầu', 'Mai bận học rồi', 'Mai học ca 3', 'Còn chết liền']


#Word2Vec

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
questions_tokenize = [nltk.word_tokenize(sent.lower()) for sent in questions]
answers_tokenize = [nltk.word_tokenize(sent.lower()) for sent in answers]

In [ ]:
print(len(questions_tokenize))
print(len(answers_tokenize))

610
610


In [ ]:
# train_text_tokenized = [nltk.word_tokenize(sent) for sent in X_train+y_train]
# test_text_tokenized = [nltk.word_tokenize(sent) for sent in X_test+y_test]

In [ ]:
from gensim.models import Word2Vec
modelw2v = Word2Vec(sentences=questions_tokenize+answers_tokenize, size=100, window=5, sg=1, min_count=1)

In [ ]:
print("Vocab size: " + str(len(modelw2v.wv.vocab)))

Vocab size: 1117


In [ ]:
modelw2v.wv.most_similar('bạn')

[('bè', 0.9991463422775269),
 ('của', 0.9991190433502197),
 ('mình', 0.9991083741188049),
 ('không', 0.9990981221199036),
 ('có', 0.999093234539032),
 ('?', 0.9990821480751038),
 ('thân', 0.9990507364273071),
 ('nhiều', 0.9990197420120239),
 ('một', 0.9990063905715942),
 ('xinh', 0.9989845752716064)]

In [ ]:
import numpy as np
temp_X = {}
error_idx = set()
for i,sent in enumerate(questions_tokenize):
  sent_tokenize = []
  for word in sent:
    try:
      sent_tokenize.append(modelw2v.wv[word])
    except Exception:
      continue
  try:
    temp_X[i] = sum(sent_tokenize)/len(sent_tokenize)
  except Exception:
    error_idx.add(i)
    continue

temp_Y = {}

for i,sent in enumerate(answers_tokenize):
  sent_tokenize = []
  for word in sent:
    try:
      sent_tokenize.append(modelw2v.wv[word])
    except Exception:
      continue
  try:
    temp_Y[i] = sum(sent_tokenize)/len(sent_tokenize)
  except Exception:
    error_idx.add(i)
    continue

X = []

for e in temp_X:
  if e in error_idx:
    continue
  X.append(temp_X[e])

Y = []

for e in temp_Y:
  if e in error_idx:
    continue
  Y.append(temp_Y[e])


print(np.array(X).shape)
print(np.array(Y).shape)
print(X[0:2])
print(Y[0:2])

(599, 100)
(599, 100)
[array([-0.14450935, -0.20649293, -0.09451679,  0.0890522 ,  0.02899261,
        0.05026393,  0.05034666, -0.06282055, -0.24767008, -0.17239507,
       -0.10587132,  0.01976674, -0.04990622,  0.03998885, -0.02010054,
       -0.06187199, -0.07163624,  0.07782615,  0.12685637,  0.07072504,
       -0.13652346, -0.09435675,  0.04682611,  0.05647584,  0.00275152,
       -0.09343237,  0.02385063, -0.06991659,  0.12345809, -0.04194152,
       -0.14403842, -0.1453831 , -0.1135772 , -0.00553429,  0.00731499,
       -0.02987967,  0.00663317, -0.09426998,  0.04331294, -0.18051247,
        0.09915093, -0.05769233, -0.02216022, -0.00825734, -0.06592613,
       -0.0688859 ,  0.0044343 , -0.05598598,  0.13394482,  0.07580458,
       -0.0286034 , -0.1659893 , -0.14259553, -0.01678232,  0.14705017,
        0.0989581 ,  0.03352637,  0.10133636, -0.14617236, -0.03517994,
       -0.06195349, -0.09838831,  0.11858205, -0.16638839,  0.02413272,
        0.01955777, -0.02946651, -0.04793

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# from keras.utils import to_categorical

# y_train = to_categorical(train_labels)
# y_test = to_categorical(test_labels)

In [ ]:
from keras.models import Sequential
from keras import layers

# # LSTM
# lstm = Sequential()

# w2v_weights = modelw2v.wv.vectors

# print(w2v_weights.shape[0])
# print(w2v_weights.shape[1])

# lstm.add(layers.Embedding(input_dim=w2v_weights.shape[0], output_dim=w2v_weights.shape[1]))
# lstm.add(layers.Bidirectional(layers.LSTM(units=300)))
# lstm.add(layers.Dense(600, activation='relu'))
# lstm.add(layers.Dense(300, activation='relu'))
# lstm.add(layers.Dense(900, activation='relu'))
# lstm.add(layers.Dense(w2v_weights.shape[0], activation='softmax'))
# lstm.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# lstm.summary()

In [ ]:
model = Sequential()
model.add(layers.LSTM(300, input_shape=(1117,100)))
model.add(layers.RepeatVector(100))
model.add(layers.LSTM(300, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(64)))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 300)               481200    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 100, 300)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 300)          721200    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 64)           19264     
Total params: 1,221,664
Trainable params: 1,221,664
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10


ValueError: ignored